<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex4_fatemeh_dracula_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4


Per group create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)

1.   Change the data source from "frankenstein.txt" to "dracula.txt", 
2.   Train the network for 10 epochs and test it on the 3 example sentences
3.   Write down 3 more example sentences and test the model on these as well
4.   Train the LSTM for 50 more epochs and run the completion again. What do you observe?
4.   Add another LSTM layer to our RNN model
5.   Train the 2-layer LSTM for 10 epochs and see how the sentence completion has changed
6.   Change the "dracula.txt" to "trump.txt", which contains Tweets from Donald Trump from 2019 and 2020.
7.   Train a single-layer LSTM on the Tweet's from Trump. If you run out of memory while preprocessing the data, change ```step = 3``` to ```step = 10```.
8.   Run the completion on the six sentences from before again. What do you observe?



**Question**
The models make very few spelling errors and few grammar error, but the content is mostly nonsense. Why do you think are the error types so different?

We will meet in the general channel at 12:10!

In [18]:
! nvidia-smi

Fri Oct 30 11:00:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |    645MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
import tensorflow as tf
import numpy as np

# Downloading the data

In [3]:
filename = "dracula.txt"  
path = tf.keras.utils.get_file(
    filename, origin=f"https://pub.ist.ac.at/~mlechner/datasets/{filename}"
)
with open(path, encoding="utf-8") as f:
    text = f.read().lower()           # Make lowercase 
text = text.replace("\n", " ")        # Remove line-breaks & newlines
print("Corpus length:", len(text))

860160/857524 [==============================] - 1s 1us/step
Corpus length: 842159


In [4]:
# Print the first 100 characters of the file we downloaded
print("First 100 characters: ",text[:100])
print("Last 100 characters : ",text[-100:])

First 100 characters:  dracula, by bram stoker  chapter i  jonathan harker's journal  (_kept in shorthand._)   _3 may. bist
Last 100 characters :  will understand how some men so loved her, that they did dare much for her sake."  jonathan harker. 


# Define input features and output labels

In [5]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

Total chars: 64
Number of sequences: 280703


In [6]:
print("First sentence:")
print(sentences[0])
print("First label: ")
print(next_chars[0])

First sentence:
dracula, by bram stoker  chapter i  jonathan harke
First label: 
r


# Transforming text into numerical data

In [7]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros(len(sentences), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]

In [8]:
# First training samples
print(x[0])
print("label: ")
print(y[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
label: 
42


# Splitting training and validation data

In [9]:
# We can define our training and validation dataset as simple numpy arrays
split_index = int(0.2*x.shape[0])
valid_x, valid_y = x[-split_index:],y[-split_index:]
train_x, train_y = x[:-split_index],y[:-split_index]
print("valid_x.shape",valid_x.shape)
print("valid_y.shape",valid_y.shape)
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)

valid_x.shape (56140, 50, 64)
valid_y.shape (56140,)
train_x.shape (224563, 50, 64)
train_y.shape (224563,)


# Defining our Recurrent Neural Network

In [10]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(maxlen, len(chars))),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["sparse_categorical_accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
Total params: 107,072
Trainable params: 107,072
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
training_log = model.fit(train_x, train_y, batch_size=64, epochs=50, validation_data=(valid_x,valid_y))

Epoch 1/50
3509/3509 [==============================] - 20s 6ms/step - loss: 1.8259 - sparse_categorical_accuracy: 0.4656 - val_loss: 1.6234 - val_sparse_categorical_accuracy: 0.5201
Epoch 2/50
3509/3509 [==============================] - 19s 5ms/step - loss: 1.5550 - sparse_categorical_accuracy: 0.5376 - val_loss: 1.5580 - val_sparse_categorical_accuracy: 0.5358
Epoch 3/50
3509/3509 [==============================] - 19s 5ms/step - loss: 1.4926 - sparse_categorical_accuracy: 0.5547 - val_loss: 1.5252 - val_sparse_categorical_accuracy: 0.5458
Epoch 4/50
3509/3509 [==============================] - 19s 5ms/step - loss: 1.4593 - sparse_categorical_accuracy: 0.5634 - val_loss: 1.5210 - val_sparse_categorical_accuracy: 0.5506
Epoch 5/50
3509/3509 [==============================] - 19s 5ms/step - loss: 1.4402 - sparse_categorical_accuracy: 0.5680 - val_loss: 1.5023 - val_sparse_categorical_accuracy: 0.5571
Epoch 6/50
3509/3509 [==============================] - 19s 5ms/step - loss: 1.4238 -

# Let's try out our model

In [15]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
def complete_sentence(sentence):
  if len(sentence) < maxlen:
    raise ValueError(f"Sentence must have at least {maxlen} characters, but provided sentence had only {sentence}")
  if len(sentence) > maxlen:
    sentence = sentence[-maxlen:]
  generated = ""
  print(f"Continuing sentence '{sentence}' ...")

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  print("... ", generated)
  print()

In [16]:
complete_sentence("this is an example to show how the machine would continue")

Continuing sentence ' an example to show how the machine would continue' ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...   aarngh  k e w n renia cste  e  o m li tr in er sgheaoh s  h a o it i bs   h p   io h thg t lsnnt  th w oe a cot  a h e  ohe uur  wan st t at oh dtronat htr yo  wi  hehe  h a ihe ie s  a auan anr t o 



In [ ]:
### TODO: Add 3 more sentences to complete. Think about what types of sentences would be interesting to test

# Biased data -> biased model

In [17]:
complete_sentence("    was the thiefs answer, to which the man replied ")
complete_sentence("    was the thiefs answer, to which the woman replied ")

Continuing sentence '  was the thiefs answer, to which the man replied ' ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...  b  a  t.n w ot  ntoeh rah ad "tre  h wh p  h  t te t  hhia,  bo m be "ee  of i h te aan  rile aht tl lr ois o iatan t ne   nda  scov aa    f ht  atn  yuhth o a un t otn u  ks s b t oe  wt  hss  wh nth

Continuing sentence 'was the thiefs answer, to which the woman replied ' ...
...  a th or  no  w im i t o r tt aa aei  at o uoueee-  att t  ao  udo she vic aa tic  th to oe  tht he t m  th sha a h ohr t et   t fnle a a i e  ih ih- ald t aaa  t ; ain t  a sth t  an h   w tat  s e c 

